In [1]:
# for testing the performance of Tensor Random Projection on Recommendation System
from tr_functions import *
import timeit
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# load data
ratings = pd.read_csv(r'C:\Users\LHY\Dropbox\Gunosy\Jupyter\data\ml-hetrec\user_ratedmovies-timestamps.dat', sep=None,  engine='python')
# tags = pd.read_csv(r'C:\Users\LHY\Dropbox\Gunosy\Jupyter\data\ml-25m/tags.csv', engine='python')
#ratings = ratings.sort_values("timestamp", inplace=True, ascending=True)
ratings = ratings.sort_values("timestamp", inplace=False, ascending=True)[370000:]

def ml_ratings_info(ratings):
    user_count = ratings.iloc[:,0].value_counts()
    movie_count = ratings.iloc[:,1].value_counts()
    print('Original data:')
    print('number of rating is', ratings.shape[0])
    print('number of user is', user_count.shape[0])
    print('number of movie is', movie_count.shape[0])
    print('ratings are from', datetime.fromtimestamp(ratings.iloc[:,3].min()/1000), 'to', datetime.fromtimestamp(ratings.iloc[:,3].max()/1000))
    print('time gap is', (ratings.iloc[:,3].max() - ratings.iloc[:,3].min())/365/24/3600/1000,'years')
#print('data sparsity is', tags_count.shape[0]tags.iloc[:,0].value_counts().shape[0])
ml_ratings_info(ratings)
ratings[0:3]

Original data:
number of rating is 485598
number of user is 2111
number of movie is 9544
ratings are from 2005-12-27 09:06:02 to 2009-01-05 13:32:20
time gap is 3.0279039193302895 years


,userID,movieID,rating,timestamp
569876,46085,1407,3.5,1135641962000
569986,46085,2858,3.5,1135642017000
569988,46085,2890,4.5,1135642044000


In [3]:
# filtering users
ratings_select = ratings.copy()
num = 20

user_count = ratings_select.iloc[:,0].value_counts()
for i in range(user_count.shape[0]):
    if np.array(user_count)[i] < num:
        ratings_select = ratings_select[~ratings_select['userID'].isin([user_count.index[i]])]
        
user_count = ratings_select.iloc[:,0].value_counts()
print(user_count[user_count.shape[0]-3:])
ml_ratings_info(ratings_select)

27935    20
59995    20
54250    20
Name: userID, dtype: int64
Original data:
number of rating is 485030
number of user is 2045
number of movie is 9544
ratings are from 2005-12-27 09:06:02 to 2009-01-05 13:32:20
time gap is 3.0279039193302895 years


In [4]:
# filtering movies
movie_count = ratings_select.iloc[:,1].value_counts()
for i in range(movie_count.shape[0]):
    if np.array(movie_count)[i] < num:
        ratings_select = ratings_select[~ratings_select['movieID'].isin([movie_count.index[i]])]
        
movie_count = ratings_select.iloc[:,1].value_counts()
print(movie_count[movie_count.shape[0]-3:])
ml_ratings_info(ratings_select)

8982     20
27022    20
2247     20
Name: movieID, dtype: int64
Original data:
number of rating is 447535
number of user is 2045
number of movie is 3696
ratings are from 2005-12-27 09:06:02 to 2009-01-05 13:32:20
time gap is 3.0279039193302895 years


In [5]:
# re-allocate the index of data
ratings_select.userID = ratings_select.userID.astype('category')
ratings_select.movieID = ratings_select.movieID.astype('category')
ratings_select.timestamp = ratings_select.timestamp.astype('category')
ratings_select['userIndex'] = ratings_select.userID.cat.codes
ratings_select['movieIndex'] = ratings_select.movieID.cat.codes
ratings_select['timeIndex'] = ratings_select.timestamp.cat.codes

ratings_select = ratings_select.sort_values("timeIndex", inplace=False, ascending=True)
ratings_select[0:3]

,userID,movieID,rating,timestamp,userIndex,movieIndex,timeIndex
569876,46085,1407,3.5,1135641962000,1319,651,0
569986,46085,2858,3.5,1135642017000,1319,1251,1
569988,46085,2890,4.5,1135642044000,1319,1264,2


In [6]:
# time segmentation by month
seq = ratings_select['timeIndex'].max()//36
time_seg = []
for i in range(ratings_select.shape[0]):
    if ratings_select.iloc[i,6]//seq != 36:
        time_seg.append(ratings_select.iloc[i,6]//seq)
    else:
        time_seg.append(35)
ratings_select.insert(7, 'timeSeg', time_seg)
ratings_select[0:3]

,userID,movieID,rating,timestamp,userIndex,movieIndex,timeIndex,timeSeg
569876,46085,1407,3.5,1135641962000,1319,651,0,0
569986,46085,2858,3.5,1135642017000,1319,1251,1,0
569988,46085,2890,4.5,1135642044000,1319,1264,2,0


In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(ratings_select[['userIndex', 'movieIndex', 'timeSeg']].values, 
                                                    ratings_select['rating'].values, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape,y_train.shape, y_test.shape)

(358028, 3) (89507, 3) (358028,) (89507,)


In [8]:
# save processed data
ratings_select.to_csv(r'C:\Users\LHY\Dropbox\Gunosy\Jupyter\data\hetrec_20core_explicit.csv') 

In [9]:
# train on the training set

# TR-SGD

tensor_size = [ratings_select['userIndex'].max()  + 1,ratings_select['movieIndex'].max() + 1, 36]
print('tensor size is', tensor_size)

tr_rank = [10, 10, 10]

start1 = timeit.default_timer()
cores = TR_SGD(X_train, y_train, tensor_size, tr_rank, learning_rate = 0.01, epoch = 2, alpha = 0.01, beta = 1)
stop1 = timeit.default_timer()
print('spend time is', stop1 - start1)

y_approx = []
for i in range(X_train.shape[0]):
    y_approx.append(np.trace(cores_multi(cores, np.array(X_train[i]), remove_n = None)))
MAE = evaluation_MAE(y_train, y_approx)
print('order-3 MAE on training set is', MAE)

tensor size is [2045, 3696, 36]
epoch 1 finish. RSE of random 100 samples is 0.2174594464302721
epoch 2 finish. RSE of random 100 samples is 0.23248218129839573
spend time is 43.22656809999998
order-3 MAE on training set is 0.6065236104555491


In [10]:
# test on the test set
y_test_approx = []
for i in range(X_test.shape[0]):
    y_test_approx.append(np.trace(cores_multi(cores, np.array(X_test[i]), remove_n = None)))
MAE = evaluation_MAE(y_test, np.array(y_test_approx).T)
print('order-3 MAE on test set is', MAE)

order-3 MAE on test set is 0.6364672868326426


In [11]:
y_pred = np.array(y_test_approx).T
print('real rating:',y_test[0:5])
print('pred rating:',y_pred[0:5])

real rating: [4.5 3.5 4.  3.  3.5]
pred rating: [4.25694433 3.02635601 2.47200297 3.50731822 4.50153581]
